In [1]:

import itertools
get_ipython().magic(u'matplotlib inline')
import numpy as np
import matplotlib
%matplotlib qt5
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import vertex_model as model
from vertex_model.run_select import run_simulation_INM, definecolors
import vertex_model.initialisation as init

In [2]:
import vertex_model
print("test")


test


In [2]:
#### Parameter used in the simulation are picked up form Gobal_Constant.py file

### Choose simulation to run
# def run_simulation_INM(x, timend,rand, type):
#     sim_type 0 simulation_with_division_clone (no differentiation rate)
#     sim_type 1 simulation_with_division_clone_differentiation (all differentiation rate)
#     sim_type 2 simulation_with_division_clone_differenciation_3stripes (2 population with and without diffentiation rate)
#     sim_type 3 simulation_with_division_clone_whole_tissue_differenciation (differentiation rate everywhere)

#use type 2 and parameter i=4 for floor plate
import numpy as np
type_=2
L_point = [-.4, -0.2,-0.3, -0.05, 0.075, 0.15]
G_point = [0.14, 0.12, 0.1, 0.065, 0.04, 0.02]
i=4
G=G_point[i]
L=L_point[i]
K=1.0
#run simulation with the choosen parameters ; line 521 in run_select.py
rand =  np.random.RandomState() #random number to choose Lambda
params = [K,G,L]  # K=x[0],G=x[1],L=x[2]
# Commented by Rebeca:
#history = run_simulation_INM(params,300,rand,type_) #return history = a list of length 300; simulation time = 300
# history = a list of cells objects at all timepoints in the simulation

# Run simulation multiple times - (17/04)
no_simulations = 5
Cell_cycle_lengths = []
for i in range(no_simulations):
    history = run_simulation_INM(params,300,rand,type_)  
    last = history[-1] # get state of the cells object from the previous step
    deadcells_last = np.where(last.empty())[0] # mask for deadcells at the last timestep
    new_cell_cycle_lengths = last.properties['age'][deadcells_last] # correct cell cycle lengths = age of deadcells
    Cell_cycle_lengths = np.append(Cell_cycle_lengths, new_cell_cycle_lengths)
    

# Code above works.

0.001
0.001
0.001
0.001
0.001


In [158]:
# 16/04

# evolution of position for ONE nucleus -> IKNM model 1 and 2
D=0.00002
k = 0.015 # a constant
timend=300 # length of simulation
dt = 0.001
final_zt_test = []

zt_initial = 0
for t in np.arange(0, 300, dt):
    zt_initial = zt_initial + np.sqrt(2*D*dt)*np.random.randn(1) # random number between 0 and 1
   

    final_zt_test.append(zt_initial)

time = np.arange(0, 300, dt)

plt.plot(time, final_zt_test)

plt.show()


    


In [282]:
# FINALLY CORRECT
last = history[-1]
print(len(cells.properties['age'])) # 3474 total cells = 1537 deadlcells + 1937 alive cells ACROSS ENTIRE TIMESPAN.
print(len(last.properties['age'])) # 23132 cells in total at the last timestep - correct
deadcells_last = np.where(last.empty())[0] # mask for deadcells at the last timestep
new_cell_cycle_lengths = last.properties['age'][deadcells_last] # correct cell cycle lengths = age of deadcells
print(len(new_cell_cycle_lengths)) # 11379 deadcells at last timestep
print(type(new_cell_cycle_lengths))
new_alivecells = np.where(~last.empty())[0] #  alive cells at the last timestep = 11753
print(len(new_alivecells))


print(len(last)) # at the last timestep there are 23132 cells
print(len(Cell_cycle_lengths))
print(len(np.where(~cells.empty())[0])) # alivecells in total, across all timsteps
print(len(np.where(cells.empty())[0]))


3474
23132
11379
<class 'numpy.ndarray'>
11753
23132
3074
1937
1537


In [168]:
print(new_cell_cycle_lengths)


[3.85437592 3.97039189 2.93285886 ... 0.31852258 0.29116626 0.11549087]


In [283]:

new_cell_cycle_lengths_LIST = new_cell_cycle_lengths.tolist()
print(type(new_cell_cycle_lengths_LIST))
print(len(new_cell_cycle_lengths))
plt.hist(new_cell_cycle_lengths_LIST, bins = 70)
plt.show()

<class 'list'>
11379


In [307]:
# Check for gamma distribution

# CORRECT
# Fit line to histogram
import matplotlib.mlab as mlab
from scipy.stats import gamma
from scipy.stats import erlang
x = np.linspace(0,5.2,11379)

#define min of array new_cell_cycle_lengths -> name it loc -> use it in fit
loc = min(new_cell_cycle_lengths)
# Gamma
print(loc)
shape, loc, scale = gamma.fit(new_cell_cycle_lengths, loc = 1, fa=1.2) # loc = position of cutoff
print(shape, loc, scale)
y = gamma.pdf(x, shape, loc, scale) # inverse of scale = rate
h = plt.hist(new_cell_cycle_lengths, bins = 70, density=True)
plt.plot(x, y, lw=2)
plt.show()

0.11549086735962699
1.2 0.9043977947448647 0.9801754607063138


In [308]:
# Perform Kolmogorov-Smirnov to test for GAMMA distribution

from scipy.stats import kstest
kstest(Cell_cycle_lengths, 'gamma', args=(1.2, 0.9043977947448647, 0.9801754607063138)) # => GAMMA

KstestResult(statistic=0.6126438089046178, pvalue=0.0)

In [132]:
# Fit line to histogram
import matplotlib.mlab as mlab
import scipy
from scipy.stats import norm

x = np.linspace(0,5.5, 3074)

#Normal
mu, sigma = scipy.stats.norm.fit(Cell_cycle_lengths) # loc = position of cutoff
print(mu, sigma)
best_fit_line = scipy.stats.norm.pdf(40, mu, sigma)
plt.hist(Cell_cycle_lengths, bins = 40)
plt.plot(x, y*300, lw=2)
plt.show()


3.237836920697841 0.6751338155184441


In [292]:
# Gamma?
# Fit line to histogram
import matplotlib.mlab as mlab
from scipy.stats import gamma
x = np.linspace(0,5.5,3074)

# Gamma
shape, loc, scale = gamma.fit(Cell_cycle_lengths) # loc = position of cutoff
print(shape, loc, scale)
y = gamma.pdf(x, shape, loc, scale) # inverse of scale = rate
h = plt.hist(Cell_cycle_lengths, bins = 40)
plt.plot(x, y*300, lw=2)
plt.show()


213.92641178676854 -6.639527865383446 0.046171596258460096


In [116]:
# Perform Kolmogorov-Smirnov test to for GAMMA distribution

from scipy.stats import kstest
kstest(Cell_cycle_lengths, 'gamma', args=(213.92641178676854, -6.639527865383446, 0.046171596258460096)) # it is gamma.
# modify shape to 1 to check for Erlang => p-value = 0

KstestResult(statistic=0.016461684513238872, pvalue=0.371434863638903)

In [114]:
# Perform Kolmogorov-Smirnov test to test for NORMAL distribution.

from scipy.stats import kstest
mu, sigma = 3.2378, 0.6751
kstest(Cell_cycle_lengths,'norm') # => NORMAL 



KstestResult(statistic=0.9550397070942962, pvalue=0.0)

In [91]:
print(Cell_cycle_lengths)
print(len(Cell_cycle_lengths))
print(min(Cell_cycle_lengths))
print(max(Cell_cycle_lengths))
print(cell_cycle_lengths)
print(len(cell_cycle_lengths))

[2.70132884 2.24096166 2.63895108 ... 0.80100273 0.76280735 0.79074712]
3074
0.36079052102142384
3.4691207881402404
[2.607621   2.29400969 2.17832815 ... 0.80100273 0.76280735 0.79074712]
1537


In [70]:
# MY LAST ATTEMPT - IGNORE FOR NOW

import numpy as np
type_=2
L_point = [-.4, -0.2,-0.3, -0.05, 0.075, 0.15]
G_point = [0.14, 0.12, 0.1, 0.065, 0.04, 0.02]
i=4
G=G_point[i]
L=L_point[i]
K=1.0
#run simulation with the choosen parameters ; line 521 in run_select.py
rand =  np.random.RandomState() #random number to choose Lambda
params = [K,G,L]  # K=x[0],G=x[1],L=x[2]

no_simulations = 2
Cell_cycle_lengths = []
for i in range(no_simulations):
    history = run_simulation_INM(params,50,rand,type_)  # reduced time from 300 to 50
    last = history[-1] # get state of the cells object from the previous step. History is a list of cells objects at
    #sequential timesteps -> get the last time step -> history[-1]
    
    def definecolors(cells):
        peach = '#eed5b7'
        light_blue ='#87cefa'
        pink = '#ffc0cb'
        light_green = '#98fb98'
        palette = np.array([peach,'g',peach, 'r',peach,'g',peach])
        colors = cells.properties['parent_group']

        return palette[colors]


    for cells in history:
        cells.properties['color'] = definecolors(cells)
        
    fig=plt.figure()
    ax = fig.gca()
# initialization function: plot the background of each frame
    def init_fig():
        ax = plt.figure()
        return (ax,)
# animation function. This is called sequentially
    def animate_fig(i):
        cells_array=history
        v_max = np.max((np.max(cells_array[0].mesh.vertices), np.max(cells_array[-1].mesh.vertices)))
        size = 2.0*v_max
        cells = history[i]
        return model.draw(cells,ax,size)
# call the animator. blit=True means only re-draw the parts that have changed.

    anim = animation.FuncAnimation(fig,animate_fig, init_func=init_fig,frames=len(history))
    
    deadcells = np.where(cells.empty())[0] # create mask for deadcells
    cell_cycle_lengths = last.properties['age'][deadcells]
    Cell_cycle_lengths = np.append(Cell_cycle_lengths, cell_cycle_lengths)

0.001
[1400] [2580 2581 2582 4221 4222 4223]
[1998] [4080 4081 4082 7191 7192 7193]
0.001
[2064] [1626 1627 1628 7395 7396 7397]


In [82]:
# IT WORKED!!! - IGNORE 
print(Cell_cycle_lengths)
print(len(Cell_cycle_lengths))
print(min(Cell_cycle_lengths))
print(max(Cell_cycle_lengths))
print(Cell_cycle_lengths[1560]) # here starts the second cell_cycle_lengths

print(len(cell_cycle_lengths))
print((cell_cycle_lengths))
# now run with len = 300

[2.25690504 2.49519104 2.33779534 ... 1.00281598 0.92970437 0.93250162]
3097
0.3394980928018918
3.3939339529302672
2.118409687238453
1537
[2.11840969 2.38781345 2.42508161 ... 1.00281598 0.92970437 0.93250162]


In [6]:
HTML(anim.to_html5_video())

RuntimeError: Requested MovieWriter (ffmpeg) not available

In [444]:
print(len(np.where(~cells.empty())[0])) #number of alive cells
alivecells = np.where(~cells.empty())[0]
age_alivecells = cells.properties['age'][alivecells]


1345


In [449]:
# All cell cycle phases - target positions calculations

zt = cells.properties['zposn'] # current positions of cells
z0 = np.zeros_like(zt) # Return an array of zeros with the same shape and type as the nucl position array (all cells).

# G1
G1_cells = np.where(properties['age'] <= t_G1)[0] # array with indices of G1 cells
print(len(G1_cells)) # 674 cells are in G1
G1_cells_ages = last.properties['age'][G1_cells]
z0[G1_cells] = 1-1.0/t_G1*G1_cells_ages


# S
S_cells = np.where((t_G1 < properties['age']) & (properties['age'] <= t_G1+t_S))[0] # array with indices of S cells
print(len(S_cells)) # 489 cells are in S
S_cells_ages = properties['age'][S_cells]
z0[S_cells] = 0.0 # positions of S cells


# G2
G2_cells = np.where((t_G1+t_S < properties['age']) & (properties['age'] <= t_G1+t_S+t_G2))[0] # array with indices of G2 cells
print(len(G2_cells)) # 182 cells are in G2
G2_cells_ages = properties['age'][G2_cells]
z0[G2_cells] = 1.0/(t_G2)*(G2_cells_ages-(t_G1+t_S)) # positions of G2 cells

# M
M_cells = np.where(properties['age'] > t_G1+t_S+t_G2)[0] # array with indices of M cells = deadcells
print(len(M_cells)) # 945 cells are in M
M_cells_ages = properties['age'][M_cells]
z0[M_cells] = 1.0 # position of M cells



674
489
182
945


In [450]:
# all nuclei positions
#all_positions = np.concatenate(z0[G1_cells], z0[G2_cells])
list_G1_pos = z0[G1_cells].tolist()
list_S_pos = z0[S_cells].tolist()
list_G2_pos = z0[G2_cells].tolist()
list_M_pos = z0[M_cells].tolist()
list_positions = list_G1_pos + list_S_pos + list_G2_pos + list_M_pos
print(len(list_positions))
plt.hist(list_positions)
plt.show()

2290


In [ ]:
# CLEAN neighbor identification -> from LAST timepoint

last = history[-1] # cells at the last timestep are in last
print(len(last)) # number of cells at the last timestep

print(type(last.mesh.edges))
print(len(last.mesh.edges)) # objects
print(len(last.mesh.face_id_by_edge)) # Same no of cell IDs as edges because IDs are repeated for each edge
print(type(last.mesh.face_id_by_edge)) # array -> iterate over its elements.


# Make a list of edges objects so that I can iterate over it
edges_ids = last.mesh.edges.ids # -> my list from 0 to number of edges -1 
reverse_edges_ids = last.mesh.edges.reverse # the list for reverse edges - each element corresponds to the respective
# edge id


# Check that the cell IDs are repeated for each edge of that particular cell
print(last.mesh.face_id_by_edge) # print cell IDs
print(type(last.mesh.face_id_by_edge))
face_id_by_edge_LIST = last.mesh.face_id_by_edge.tolist() # convert to LIST before counting an entry's occurences.
check_no = face_id_by_edge_LIST.count(17232)
print(check_no) # = 5 => cell with ID 17232 has 5 sides = 5 neighbors

# Get cells' IDs (unique)
unique_IDs = np.unique(last.mesh.face_id_by_edge) # alive cells at last timestep
print(len(unique_IDs)) # 11753 alive cells - as found before
print(type(unique_IDs))

#Identify neighbors and number of neighbors
 # initialise with 0 neighbors' nuclei positions
for unique_ID in unique_IDs: # loop over cell IDs
        neighbors_IDs = np.unique(adjacent_edge_id) #get neigh IDs as unique 
        no_neighbors = len(neighbors_IDs)
        
        zjt = last.properties['zposn'][neighbors_IDs] # get current position of neighbors
        sum_neigh_pos = 0
        for neighbor in neighbors_IDs: 
            zjt = zjt + k*(z0 - zjt)*dt + np.sqrt(2*D*dt)*np.random.randn(len(zjt))
            sum += zjt
            
            
            



In [313]:
print(last.mesh.edges.ids)
print(last.mesh.edges.reverse)
print(last.mesh.face_id_by_edge)

[    0     1     2 ... 70515 70516 70517]
[52823 18008 14250 ... 57018 70513 57016]
[17232 12076 17231 ... 12813 23131 23130]


In [339]:
# Calculate zt = current position of the nucleus by adding noise to target position
zt = last.properties['zposn'] # position at last timestep
D=0.15
k = 0.015 # a constant
timend=300 # length of simulation

dt = 0.001

for t in np.arange(0, timend, dt):
    zt = zt + k*(z0 - zt)*dt + np.sqrt(2*D*dt)*np.random.randn(len(zt))


# Adding noise to model with velocity =     
    
    


In [ ]:
# Define mean position neighbors:
# cell has min 3 neighbors
# Define neighbors: neighbors = cells whose 2 of their vertices coincide with 2 of the cell's. store in a list, neighbors.
# n = number of neighbors: n = len(neighbors)
# Position of neighbors (nuclei) = zj: repeat np.where statements for each neighbor to find out 
sum = 0
for neighbor in np.arange(3, n+1, 1): # do this under for t in np.arange(0, timend, dt)
    zt = zt + k*(z0 - zt)*dt + np.sqrt(2*D*dt)*np.random.randn(len(zt))
    sum = sum + zt
zj = sum/n #calculate mean neighbor position
    
      
    
# Adding CROWDING
def f(zj):
    
def force_on_cell(zi,zj): #force on cell at position zi
    force = 0
    
    for zj in neighbors(zi): 
        if zi-zj < 0:
            f1 = -f1
        elif zi-zj > 0.2: # if the nuclei do not overlap - say nucleus length along apico-basal axis = 0.1
        # nuclei do not overlap if zi-zj > 2 x nucl length
            f(zi-zj) = 1/m*f(zi-zj) # make the force vanishing ~
        else :
            f(zi-zj) = f(zi-zj)
      
        force += f(zi-zj)
    
    return force

In [ ]:
# IKNM dynamics 1 -> drift = velocity from current model

#G1. if 0 <= properties['age'] <= t_G1:
v1 = -1/t_G1
#elif t_G1 < properties['age'] <= t_G1 + t_S :
v1 = 0
# elif t_G1 + t_S < properties['age'] <= t_G1 + t_S + t_G2:
v1 = 1/t_G2
# else: (t_G1 + t_S + t_G2 < properties['age'])
v1 = 0

# v2 = A*(properties['zposn'] - mean_neigh_pos)
# cell has min 3 neighbors
# Define neighbors: neighbors = cells whose 2 of their vertices coincide with 2 of the cell's. store in a list, neighbors.
# n = number of neighbors: len(neighbors)
# Position of neighbors (nuclei) = neigh_pos: repeat if statements for each neighbor to find out v1
 sum = 0
for i in range(3, n+1, 1):
sum = sum + neigh_pos
# mean neigh position: 
mean_neigh_pos = sum/n

# A = ct; value?

v = v1 + v2








In [98]:
#this is not actually used

# Euler–Maruyama method - Wikipedia code - Potentially add noise dependence on the current position -> if cell is in
# G1 or S => higher noise. But this might be counteracted by the low position of the nuclei, which are pushed further
# down by neighbors above -> this noise might be counteracted in this way so probs not worth adding dependence on pos.

num_sims = 5  # Display five runs

t_init = 3
t_end  = 7
N      = 1000  # Compute 1000 grid points
dt     = float(t_end - t_init) / N
y_init = 0

c_theta = 0.7
c_mu    = 1.5
c_sigma = 0.06

def mu(y, t):
    """Implement the Ornstein–Uhlenbeck mu."""  # = \theta (\mu-Y_t)
    # replace with v
    return c_theta * (c_mu - y)

def sigma(y, t):
    """Implement the Ornstein–Uhlenbeck sigma."""  # = \sigma
    # Diffusion coefficient
    return c_sigma

def dW(delta_t):
    """Sample a random number at each call."""
    # White Gaussian noise
    return np.random.normal(loc=0.0, scale=np.sqrt(delta_t))

ts = np.arange(t_init, t_end + dt, dt)
ys = np.zeros(N + 1)

ys[0] = y_init

for _ in range(num_sims):
    for i in range(1, ts.size):
        t = t_init + (i - 1) * dt
        y = ys[i - 1]
        ys[i] = y + mu(y, t) * dt + sigma(y, t) * dW(dt)
    plt.plot(ts, ys)

plt.xlabel("time (s)")
h = plt.ylabel("y")
h.set_rotation(0)
plt.show()

In [224]:
print(cells.empty())
print(~cells.empty()) # Turns Trues into Falses and viceversa.

[ True  True  True ... False False False]
[False False False ...  True  True  True]


In [240]:
a = np.random.normal(size=5)
print(a)
print(type(np.where(a>0)))
print(type(np.where(a>0)[0]))

[ 0.34678344  0.09469718  1.41721669  1.67473528 -0.33489884]
<class 'tuple'>
<class 'numpy.ndarray'>


In [219]:
a = np.random.normal(size=(5,3))
print(a)
print(np.where(a>0)) # first array gives rows where a>0; second array gives columns where a>0, for each row
print(np.where(a>0)[0]) # returns only the first dimension = the rows of the array where a>0, repeated for as many..
# ..times a number a>0 exists

[[ 1.217858   -1.07929465 -0.64485792]
 [ 0.94932936  0.97203389  0.25382137]
 [ 1.46529953  0.4708984   0.64353754]
 [-0.00430902  0.84851356 -0.42626672]
 [ 0.71115515 -1.30060859 -1.01177399]]
(array([0, 1, 1, 1, 2, 2, 2, 3, 4]), array([0, 0, 1, 2, 0, 1, 2, 1, 0]))
[0 1 1 1 2 2 2 3 4]
